# 交差検証とグリッドサーチ
## 前回行ったこと
 - `get_dummies`によるone-hotエンコーディング
 - 検証用データを用いたモデルの投稿前の評価

## 今回行うこと
- 交差検証とハイパーパラメータチューニングのためのクラス`GridSearchCV`の使い方
- 学習後のモデルの観察

## （復習）データの読み込み・特徴ベクトルの構築
前回の資料で構成した，one-hotエンコーディングを用いた特徴ベクトルを再び作ります．
もう詳しく説明することはしません．
詳細は前回の資料を参照してください．

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Google Colabを用いる場合、以下のコメントアウトを外して実行
# from google.colab import drive
# drive.mount('/content/drive') # google driveをマウント（＝Colabから使えるようにする）

Mounted at /content/drive


In [3]:
# Google Colabを用いる場合、以下のコメントアウトを外して実行
# d_train = pd.read_csv("drive/My Drive/data/train.csv") # 訓練データを読み込む．パスは適宜変更すること
# d_test = pd.read_csv("drive/My Drive/data/test.csv") # テストデータを読み込む．パスは適宜変更すること

In [ ]:
# ローカル環境を用いる場合、以下のコメントアウトを外して実行
# d_train = pd.read_csv("/data/train.csv") # 訓練データを読み込む．パスは適宜変更すること
# d_test = pd.read_csv("/data/test.csv") # テストデータを読み込む．パスは適宜変更すること

In [4]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")

# targetの値
y_train = d_train.pop('popularity')
y_train = y_train.to_numpy() # numpyのarrayに変換
print(y_train)

訓練データとテストデータの数を取得
訓練データ数：79800，テストデータ数：34200
[43  0 50 ... 60 44 24]


以下のセルでOne-hotエンコーディングを行い特徴ベクトルを作ります．

変数に`_one_hot`とつけるのは面倒なので，今回は`X_train`，`X_test`という変数名にします．

In [5]:
d_all = pd.concat([d_train, d_test], axis=0) # 訓練データとテストデータを連結
d_all = d_all.astype({'explicit': int}) # `explicit`の列の型をintにキャスト
columns_cat = ["track_genre"] # カテゴリカル変数の列名

d_all_onehot = pd.get_dummies(d_all, columns=columns_cat, dtype=int) # get_dummiesを使ってone-hotエンコーディング．columnsに指定された列のみone-hotエンコーディングし，出力としてint型を指定
d_train_onehot = d_all_onehot[:n_train] # d_all_onehotの訓練データ部分
d_test_onehot = d_all_onehot[n_train:] # d_all_onehotのテストデータ部分
X_train = d_train_onehot.select_dtypes(include=['int64', 'float64']).to_numpy() # 数値部分のみ取り出し，np.arrayに変換
X_test = d_test_onehot.select_dtypes(include=['int64', 'float64']).to_numpy()  # 数値部分のみ取り出し，np.arrayに変換

いい加減しつこい気もしますが，復習と予測結果の比較のために`LinearRegression`を動かしておきます．
ハイパーパラメータ（学習するのではなく，ユーザが決める要素）はここでは`fit_intercept=False`とします．

In [7]:
from sklearn.linear_model import LinearRegression # LinearRegressionを使えるようにする
lr = LinearRegression(fit_intercept=False) # インスタンスの作成
lr.fit(X_train, y_train)
y_pred_test_lr = lr.predict(X_test)
print(y_pred_test_lr)

[36.07987563 32.40019485 55.05526121 ... 45.410423   42.58279019
 40.51070228]


## 交差検証（Cross Validation）
特徴ベクトルの作り方やどの手法を使うか，さらには同じ手法であっても**ハイパーパラメータ**（バイアス項を使うか？正則化項の強さはどれくらいか？中間層のユニット数はどれくらいか？等の，ユーザが定める要素）の値よって，予測の正確さは大きく変わってきます．
前回は「ラベル付きデータを訓練用/検証用に分割し，訓練用データで学習した後，学習に使わなかった検証用で評価」ということを行いました．
しかし，訓練・検証の分割の仕方によって評価の値は当然変わってきます．
分割によっては，本当は悪い手法であるのにもかかわらず良い評価となったり，また反対に，本当は良い手法であるのにもかかわらず悪い評価となることもあるでしょう．

そこで，次のような上手い方法がしばしば行われます：データを`K`分割（`K=5,10`等）し，そのうちの`K-1`個の分割を訓練，`1`個の分割を検証データとして用いることを繰り返し，あるハイパーパラメータの時のモデルの性能を`K`回のスコア（評価値）の平均で評価する．
この方法を**（K-分割）交差検証（(K-fold) cross validation）**といいます．
K=訓練データ数のときは特にLeave-one-out交差検証（LOOCV）と呼ばれたりもします．
一回だけでなくK回の評価の平均を取っているので，前回の方法よりもより正確に性能の評価ができそうです．
一方で，学習をK回行うため，前回の方法と比べると時間はかかってしまいそうです．

今回はsklearnの便利な関数を使って，いくつかの方法で交差検証を行ってみます．

## Ridge回帰の交差検証+グリッドサーチによるハイパーパラメータチューニング
### Ridge回帰：正則化付きの線形回帰
今，特徴ベクトルと使う手法は固定して，その手法の適切なハイパーパラメータを定めるということを考えます．
前回までの例でいうと，特徴ベクトルは数値情報+one-hotエンコーディング，手法は`LinearRegression`と決めて，`LinearRegression`においてバイアス項を使うか否か（`fit_intercept`を`True`にするか`False`にするか）を定めることに対応します．

ずっと線形回帰を使うのも楽しくありませんが，かと言っていきなり複雑な手法を使うのもすこし大変ですので，今回は線形回帰に**正則化**項を加えたRidge回帰，sklearnにおける`Ridge`を用いることにします．
正則化は過学習を防ぐための方法の一つで，Ridge回帰は線形回帰の目的関数にパラメータの $L_2$ ノルムの2乗を加えたものでした（講義データサイエンスの初回の資料参照）．
線形モデルにおいて，重みの値が非常に大きい場合，入力の値がほんの少し変化しただけで予測の値が大きく変わってしまいます．
そのような場合，訓練データに対しては正しい予測ができるけれども，訓練データとほんの少し値が違うだけのデータに対しては正しい予測ができなくなってしまいます．
これはまさしく過学習です．
しかし，重みの値に対して罰則を課す正則化項を目的関数に加えることで，重みの値を過剰に大きくしてまでは訓練データにフィットしなくなり，過学習を防げます．

Ridge回帰は以下のようにしてimportできます．

In [8]:
from sklearn.linear_model import Ridge

[Ridgeのドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)を見てみましょう．
`LinearRegression`と比べるとユーザが定めることのできる要素が色々と追加されています．
`fit_intercept`は`LinearRegression`にもありました．
`alpha`がここでは最も重要で，正則化項の強さを決めるハイパーパラメータです．
`Ridge`回帰においてパラメータを定める方法（学習アルゴリズム）はいくつかあり，それは`solver`引数で指定することができます．
`max_iter`や`tol`，`random_state`は`solver`に関係する引数です．
本来はデータの数や次元数などを考慮して適切に選ぶべき（学習時間に大きく影響します）ですが一旦置いておきます．ここではとりあえず，`LinearRegression`との比較のためにsolver="svd"`とだけ定めます．

練習がてら，`alpha=10.0`，`fit_intercept=False`で動かしてみましょう．

In [9]:
ridge = Ridge(alpha=10.0, fit_intercept=False, solver="svd")
ridge.fit(X_train, y_train)
y_pred_test_ridge = ridge.predict(X_test)
print(y_pred_test_ridge)

[36.39355551 32.51309548 53.76571668 ... 46.47130878 44.77646633
 39.2655409 ]


上の`LinearRegression`の結果と比べると，やや異なる結果となっています．
正則化が正しく動いていれば，パラメータ $\mathbf{w}$ のノルムが小さくなっているはずです．
ちょっと本題とは話がそれてしまいますが，ついでにこのことを確認してみましょう．

Pythonでは，あるオブジェクト`obj`の持つ`a`というフィールドには，`obj.a`という形式でアクセスできます．
`Ridge`や`LinearRegression`のドキュメントを見てみると，重み $\mathbf{w}$ には`coef_`という名前がついていることが分かります．


   $L_2$ ノルムの計算は自分でコードを書いてもよいですが，numpyにはちゃんと（？）ノルムを計算する関数[`numpy.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)があるので使ってみましょう．`numpy.linalg.norm`では`ord`(order)パラメータによってノルムの種類を指定しますが，何も指定しないとベクトルの入力に対して $L_2$ ノルムを計算してくれるのでこれを使います．

In [10]:
norm2_lr = np.linalg.norm(lr.coef_)
norm2_ridge = np.linalg.norm(ridge.coef_)

print(f"線形回帰の重みのノルム：{norm2_lr}")
print(f"Ridge回帰の重みのノルム：{norm2_ridge}")

線形回帰の重みのノルム：377.6741303081109
Ridge回帰の重みのノルム：149.95095135145374


結果が出てきました！
線形回帰の重みのノルムと比べると，Ridge回帰の重みのノルムは小さくなっていて，正則化が働いていることがわかります．

### Ridge回帰のハイパーパラメータチューニング：`alpha`のみチューニング

より大きな`alpha`を設定すると正則化の効果が強くなり，より重みのノルムが小さくなります（演習では行いませんが，実験ではこのことを実際に確認するはずです）．
`alpha`を非常に大きな値にした場合，重みのノルムはほとんど`0`に近い値となり，訓練データに対する性能もテストデータに対する性能も差はないがどちらも非常に低い，アンダーフィッティング（under fitting）が起こってしまいます．
一方で，`alpha`の値が小さすぎると正則化の効果がほとんどなくなってしまいます．
そのため，`alpha`は大きすぎても小さすぎてもだめで，イイカンジの値を定める必要があります．

ようやく本題です．そこで，**交差検証によって適切な`alpha`の値を定める**ことにします．
`alpha`は理論上は任意の非負の値をとることができますが，コンピュータ上ではそれは不可能です．
一般に，連続値を取るハイパーパラメータは，適当に上限と下限と間隔を定め，その中で良い値を探します．
例えば，`[0.1, 0.2, 0.3, 0.4, 0.5]`であったり，`[1e-3, 1e-2, 1e-1, 1, 10, 100]`と言ったようにです．
とりあえず今回は`[1e-3, 1e-2, 1e-1, 1, 10, 100]`の範囲で探してみましょう．
また，一旦`fit_intercept`は先程と同じように`False`としておきます．

さらに，交差検証を行う場合，何分割するかを考える必要があります．
とりあえずここでは分割数`K=5`としておきましょう．


これにて行うことが定まりました．以下のようになりそうです：
  1. 訓練データを5分割し，
  2. ある分割を検証用，その他を訓練用として，`alpha`をある範囲内（ここでは`[1e-3, 1e-2, 1e-1, 1, 10, 100]`）で動かして学習・評価することを繰り返し，
  3. ある範囲内の全ての値で学習・評価が終わった後，その結果を元に最も良い`alpha`を定める．

では残りはコードを書くだけなのですが，それなりに面倒なように感じます（もちろん，「書け」と言われれば書けるとは思いますが）．
勉強のために自分で書くことは非常に大事ですが（実験では皆さんに書いてもらうはずです），ここでは偉大なるsklearnの便利な機能を利用します．
sklearnには，[cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)という関数があり，これを使うと簡単に交差検証ができます．
`cross_validate`関数を使う場合，以下の引数を指定する必要があります：
- `estimator`：交差検証によって評価をしたいインスタンス（例：`ridge`）
- `X`：訓練データ（例：`X_train`）
- `y`：訓練データの目標値（例：`y_train`）
- `cv`：分割数，整数型（例：5)
- `scoring`：評価関数，文字列（例：`"neg_mean_squared_error"`）

そして以下の情報（キー）を持った**辞書型オブジェクト（=ハッシュテーブル）**を返します：
- `fit_time`：学習にかかった時間のリスト（要素数K)
- `score_time`：評価にかかった時間のリスト（要素数K)
- `test_score`：検証用データでのスコアのリスト（要素数K）

詳しくは上のドキュメントを読んでみてください（実際はここで述べているよりも更に柔軟です）．
今回のコンペにおける評価関数は平均二乗誤差でした．sklearnには`mean_squared_error`という関数があるためこれを`scoring`に使いたいのですが，残念ながら対応していないため代わりに`neg_mean_squared_error`を使います．
これは負の平均二乗誤差です（"neg"は"negative"）．

とりあえず使ってみましょう．
`alpha=10.0`の`ridge`を検証してみます．

In [11]:
from sklearn.model_selection import cross_validate
ridge = Ridge(alpha=10.0, fit_intercept=False, solver="svd")
scores = cross_validate(ridge, X_train, y_train, cv=5,
                        scoring="neg_mean_squared_error")
print(scores)

{'fit_time': array([3.8787272 , 3.69170451, 3.09351087, 1.86862659, 1.90792036]), 'score_time': array([0.01690435, 0.01775718, 0.00661945, 0.00646496, 0.00445151]), 'test_score': array([-370.43438561, -376.62073497, -367.48155642, -376.39055146,
       -376.14835632])}


色々`print`されました．
ちょっと見にくいですね．
今は検証用データのスコアさえあれば良いので，`"test_score"`だけ取り出します．
また，5回の評価結果の平均を計算します．
`np.mean`によってリストや`np.array`に含まれる値の平均を計算できます．
また，符号が反転しているので，わかりやすさのために戻しておきましょう．

In [12]:
print(f"交差検証の5回のスコア：{-scores['test_score']}")
print(f"交差検証の平均スコア：{-np.mean(scores['test_score'])}")

交差検証の5回のスコア：[370.43438561 376.62073497 367.48155642 376.39055146 376.14835632]
交差検証の平均スコア：373.4151169555022


`alpha=10.0`の場合の結果が無事に出てきました．

次は，`alpha`を`[1e-3, 1e-2, 1e-1, 1, 10, 100]`の範囲で動かして5分割交差検証を行い，その結果において最も良い`alpha`とその交差検証のスコア（5回の検証スコアの平均）を探索してみます．
ここで，`alpha`の探索範囲のリストとして`alpha_list`を定義しておきます．
`results`は，それぞれの`alpha`における交差検証のスコアを格納するリストです．

In [13]:
alpha_list = [1e-3, 1e-2, 1e-1, 1, 10, 100] # 探索するalphaの範囲
results = []
for alpha in alpha_list:
    ridge = Ridge(alpha=alpha, fit_intercept=False, solver="svd")
    scores = cross_validate(ridge, X_train, y_train, cv=5,
                            scoring="neg_mean_squared_error")
    results.append(-np.mean(scores["test_score"])) # resultsに結果を追加

result_best = np.min(results) # 最善のスコア
alpha_best = alpha_list[np.argmin(results)] # 最善のスコアを出した時のalphaの値(numpy.argminは最小値のインデックスを取得)
print(f"最良のalpha：{alpha_best}　交差検証スコア：{result_best}")

最良のalpha：0.001　交差検証スコア：369.94445475190776


結果が出てきたでしょうか？
私の環境では，`alpha=0.001`が（この範囲では）最もよく，またその時のスコアは`369.94445475190776`となっています．

### Ridge回帰のハイパーパラメータチューニング：`alpha`と`fit_intercept`のチューニング

今度は`alpha`だけでなく`fit_intercept`もチューニングしてみましょう．
チューニングするハイパーパラメータが複数ある場合によく行われる方法は，全てのパラメータについて探索範囲を定め，その範囲の全ての組合せを試して（=範囲の直積上で探索して），その中で最も良いハイパーパラメータの組合せを探す，というものです．
これを**グリッドサーチ**といいます．
例えば，`alpha`を`[1, 10, 100]`，`fit_intercept`を`[True, False]`の範囲で探索する場合，
 - `alpha=1`，`fit_intercept=True`
 - `alpha=1`，`fit_intercept=False`
 - `alpha=10`，`fit_intercept=True`
 - `alpha=10`，`fit_intercept=False`
 - `alpha=100`，`fit_intercept=True`
 - `alpha=100`，`fit_intercept=False`

の合計 $3\times2=6$ 通りを試します．


では実際に行ってみようと思います……が，はっきり言ってやや面倒に感じます．
なんと素晴らしいことに，sklearnにはこれを簡単に行う`GridSearchCV`というクラスが提供されています．
`GridSearchCV`は先程の`cross_validate`（関数）とは異なりクラスです．
`GridSearchCV`のインスタンスを作成する際に，ハイパーパラメータチューニングを行いたいモデルとハイパーパラメータの探索範囲を与えます．
そして，作成した`GridSearchCV`のインスタンスの`fit`メソッドを実行すると，交差検証を行い最も良いハイパーパラメータを見つけてくれます．
`GridSearchCV`のインスタンスを作成する際に，最低限指定しなければならない引数を説明します．
 - `estimator`：探索・評価をしたいモデル（例：`ridge`）
 - `param_grid`：探索を行うハイパーパラメータとその範囲，辞書型（keyがハイパーパラメータの名前（文字列），valueが探索範囲（リスト））．
 - `scoring`：評価関数，文字列（例："neg_mean_squared_error"）
 - `cv`：分割の数，整数．

詳しくは[GridSearchCVのドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)参照してください．
これもやはり実際は更に柔軟です．
注意が必要なのが，「最も良い」の意味です．
`GridSearchCV`では，`scoring`は「モデル（予測結果）の良さ」を表す関数であるとしています．
つまり，**スコアの最も高い**ハイパーパラメータを返します．
`GridSearchCV`では，`scoring`に文字列ではなく自身の作成した関数を渡すこともできますが，うっかり二乗誤差のように「モデル（予測結果）の悪さ」を表す関数を渡すと最も悪いハイパーパラメータが返ってきます．
`scoring`に`"mean_squared_error"`が指定できず，`"neg_mean_squared_error"`が用意されている理由はこの仕様のためです．

文章で説明してもわかりにくいと思いますので，とりあえず`fit_intercept`の探索範囲を`[True, False]`，`alpha`の探索範囲を先ほどと同じ範囲として動かしてみます．

In [14]:
from sklearn.model_selection import GridSearchCV

estimator = Ridge(solver="svd") # 探索するモデルを決める
param_grid = { # 探索するハイパーパラメータと範囲を決める
    "alpha": alpha_list,
    "fit_intercept": [True, False]
} # solverやmax_iterはここでは指定されていないので，ずっと同じ値

cv = GridSearchCV(estimator=estimator, # estimator引数でどのモデルを使うかを指定
                  param_grid=param_grid, # param_grid引数で探索するハイパーパラメータと範囲を指定
                  scoring='neg_mean_squared_error', cv=5) # scoringで評価指標を，cvで分割数を指定
cv.fit(X_train, y_train) # 交差検証を実行

GridSearchCV(cv=5, estimator=Ridge(solver='svd'),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
                         'fit_intercept': [True, False]},
             scoring='neg_mean_squared_error')

交差検証が終了すると，当然，その結果が知りたいです．
`fit`実行後の`GridSeachCV`は，
1. 交差検証による評価で最も良かったハイパーパラメータ：`best_params_`
2. 最も良かったハイパーパラメータを用いたときのスコア：`best_score_`
3. 最も良かったハイパーパラメータを用いたときのモデル：`best_estimator_`

を持っています．
最も良かったハイパーパラメータとその交差検証スコアを確認し，さらに`best_estimator_`で予測した結果を保存してみましょう．

In [15]:
print(cv.best_params_) # 最も良かったハイパーパラメータを見てみる
print(-cv.best_score_) # 最も良かったハイパーパラメータの時のスコアを見てみる．わかりやすさのため符号は反転する．
y_pred_ridge_gridcv = cv.best_estimator_.predict(X_test)# 最も良かったモデルで推定する．実はcv.predictでも良い
np.savetxt(X=y_pred_ridge_gridcv, fname='y_pred_ridge_cv.csv')
print(y_pred_ridge_gridcv)

{'alpha': 1, 'fit_intercept': True}
369.9420668019845
[36.08183409 32.40004005 55.02918037 ... 45.39194558 42.56519461
 40.49837409]


結果が出てきました．
上の探索範囲ではこのモデルが最も良いと考えられるので，この結果を投稿してみると良いかもしれません（しなくても良いです）．

## まとめ
- K分割交差検証：ラベル付きデータをK分割し，K-1個で学習，残りの一個で評価をK回繰り返し，手法を評価する．
- グリッドサーチ：それぞれのハイパーパラメータの探索範囲を定めて，全ての組合せを試して良いハイパーパラメータを決める．
- `GridSearchCV`を使うとハイパーパラメータチューニングが簡単にできる．

今回はRidge回帰を取り上げましたが，予測手法は他にも多数あります．
Ridge回帰や線型回帰は**非常に単純なモデル**で，まず試すべきものの一つかとは思いますが，非線形でより複雑なモデルの方が良い性能であることが多々あります．
他の手法であっても，`fit`で学習し`predict`で予測することは変わらないですし，`GridSearchCV`の使い方も同じです（勿論，ハイパーパラメータの名前や有効な範囲がわかってないといけないため，ドキュメントを読む必要は間違いなくありますが）．
是非いろいろ試してみてください．
また，sklearn以外にも機械学習ライブラリはありますので，興味がある方は調べてみると良いかもしれません．
sklearnには様々な手法が実装されていますが，特定の手法に特化したライブラリが他にいくつもあります．

グリッドサーチではハイパーパラメータの候補点をそれぞれの探索範囲の直積から取ってきましたが，候補点をランダムに決める（＝グリッドになっていない）ランダムサーチ（`RandomizedSearchCV`）であったり（少数のハイパーパラメータの影響が強い場合に有効），ハイパーパラメータに関しても最適化する（＝もっと賢くハイパーパラメータを探す）方法も提案されています．
後者についてはsklearnにはありませんが，他にライブラリがあるので，興味がある方は調べてみると良いかもしれません（一回の学習が重たい手法では，少ない学習回数で良いハイパーパラメータを見つけたいため，ハイパーパラメータチューニングの手法が重要になってきます）．
もっとレイヤーが上がったところだと，「どの手法を使うか」「どのような特徴ベクトルを作るか」というところも上手くやる，**Automated Machine Learning（AutoML）**というものもあり，AutoMLのライブラリもあったりします．
興味がある方はこちらも調べてみると面白いかもしれません．